In [ ]:
!wget -O UCF101TrainTestSplits-RecognitionTask.zip https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip

--2020-06-07 00:22:03--  https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113943 (111K) [application/zip]
Saving to: ‘UCF101TrainTestSplits-RecognitionTask.zip’

UCF101TrainTestSpli 100%[===================>] 111.27K   294KB/s    in 0.4s    

2020-06-07 00:22:05 (294 KB/s) - ‘UCF101TrainTestSplits-RecognitionTask.zip’ saved [113943/113943]



In [ ]:
!wget -O UCF101.rar https://www.crcv.ucf.edu/data/UCF101/UCF101.rar

--2020-06-07 00:22:07--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘UCF101.rar’

UCF101.rar          100%[===================>]   6.46G  12.7MB/s    in 7m 30s  

2020-06-07 00:29:38 (14.7 MB/s) - ‘UCF101.rar’ saved [6932971618/6932971618]



In [ ]:
!unzip UCF101TrainTestSplits-RecognitionTask.zip

Archive:  UCF101TrainTestSplits-RecognitionTask.zip
   creating: ucfTrainTestlist/
  inflating: ucfTrainTestlist/classInd.txt  
  inflating: ucfTrainTestlist/testlist01.txt  
  inflating: ucfTrainTestlist/testlist02.txt  
  inflating: ucfTrainTestlist/testlist03.txt  
  inflating: ucfTrainTestlist/trainlist01.txt  
  inflating: ucfTrainTestlist/trainlist02.txt  
  inflating: ucfTrainTestlist/trainlist03.txt  


In [ ]:
!unrar e UCF101.rar Videos/

Streaming output truncated to the last 5000 lines.
Extracting  Videos/v_PlayingFlute_g25_c04.avi                             61%  OK 
Extracting  Videos/v_PlayingFlute_g25_c05.avi                             61%  OK 
Extracting  Videos/v_PlayingFlute_g25_c06.avi                             61%  OK 
Extracting  Videos/v_PlayingFlute_g25_c07.avi                             61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c01.avi                            61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c02.avi                            61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c03.avi                            61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c04.avi                            61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c05.avi                            61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c06.avi                            61%  OK 
Extracting  Videos/v_PlayingG

In [ ]:
rm UCF101TrainTestSplits-RecognitionTask.zip

In [ ]:
rm UCF101.rar

In [ ]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

Using TensorFlow backend.


In [ ]:
# open the .txt file which have names of training videos
f = open("ucfTrainTestlist/trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1


In [ ]:
# open the .txt file which have names of test videos
f = open("ucfTrainTestlist/testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


In [ ]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

In [ ]:
!mkdir train_1

In [ ]:
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('Videos/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename ='train_1/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|██████████| 9537/9537 [08:23<00:00, 18.96it/s]


In [ ]:
# getting the names of all the images
images = glob("train_1/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i].split('/')[1])
    # creating the class of image
    train_class.append(images[i].split('/')[1].split('_')[1])
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('train_new.csv',header=True, index=False)

100%|██████████| 73844/73844 [00:00<00:00, 762304.17it/s]


In [ ]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('train_new.csv')
train.head()

,image,class
0,v_BasketballDunk_g23_c03.avi_frame0.jpg,BasketballDunk
1,v_SalsaSpin_g19_c01.avi_frame10.jpg,SalsaSpin
2,v_PlayingSitar_g17_c05.avi_frame1.jpg,PlayingSitar
3,v_FloorGymnastics_g15_c03.avi_frame6.jpg,FloorGymnastics
4,v_HorseRace_g11_c06.avi_frame3.jpg,HorseRace


In [ ]:

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(train['image'], train['class'], random_state=42, test_size=0.2, stratify = train['class'])
X_train.head()
X_test.head()
y_train.head()
y_test.head()

52086    FieldHockeyPenalty
69542        SoccerJuggling
19155          PlayingFlute
29950        WritingOnBoard
14763             GolfSwing
Name: class, dtype: object

In [ ]:
import keras

class My_Custom_Generator(keras.utils.Sequence) :
  
  def __init__(self, image_files, labels, batch_size) :
    self.image_files = image_files
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_files) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_files[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

    train_image = []

    for file_name in batch_x:
      img = image.load_img('train_1/'+file_name, target_size=(224,224,3))
      # converting it to array
      img = image.img_to_array(img)
      # normalizing the pixel value
      img = img/255
      # appending the image to the train_image list
      train_image.append(img)
      
    return np.array(train_image), np.array(batch_y)

In [ ]:
batch_size = 128

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

my_training_batch_generator = My_Custom_Generator(X_train, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_test, y_test, batch_size)

In [ ]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top = False, input_shape=(224, 224, 3))
base_model.summary()

58892288/58889256 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [ ]:
#defining the model architecture
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)              

In [ ]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
print(len(X_train))

59075


In [ ]:
model.fit_generator(generator=my_training_batch_generator,
                   steps_per_epoch = int(len(X_train) // batch_size),
                   epochs = 200,
                   verbose = 1,
                   validation_data = my_validation_batch_generator,
                   validation_steps = int(len(X_test) // batch_size), callbacks = [mcp_save])

Epoch 1/200
461/461 [==============================] - 442s 959ms/step - loss: 4.5623 - accuracy: 0.0208 - val_loss: 4.5302 - val_accuracy: 0.0236
Epoch 2/200
461/461 [==============================] - 427s 927ms/step - loss: 4.5387 - accuracy: 0.0227 - val_loss: 4.5538 - val_accuracy: 0.0225
Epoch 3/200
128/461 [=======>......................] - ETA: 4:43 - loss: 4.5360 - accuracy: 0.0231